# Plug-and-Play Demo: Run–Tumble with Spec-Declared 1‑Back Observation

This notebook mimics an external project that imports `plume_nav_sim` as a library and
defines its own policy. It showcases:

- Spec‑first composition using `SimulationSpec` + `prepare()`
- Dotted‑path policy import from a separate package (`plug_and_play_demo`)
- Spec‑declared observation wrapper: core `ConcentrationNBackWrapper(n=2)` so the policy sees `[c_prev, c_now]`
- Runner streaming with RGB frames (headless‑safe display)


## Capture Goals and Workflow

This demo focuses on minimal plug-and-play execution. For reproducible, analysis-ready datasets, use the data capture pipeline and CLI.

- See README: plug-and-play-demo/README.md (Capture Workflow)
- Data catalog and schemas: src/backend/docs/data_catalog_capture.md, src/backend/docs/data_capture_schemas.md

Quick start (CLI):

```bash
plume-nav-capture --output results --experiment demo --episodes 2 --grid 8x8 --parquet
```

Validate artifacts:

```python
from pathlib import Path
from plume_nav_sim.data_capture.validate import validate_run_artifacts
report = validate_run_artifacts(Path("results/demo/<run_id>"))
```


## Bundled movie plume asset

The plug-and-play demo ships with a canonical movie file
`assets/gaussian_plume_demo.avi`. When you point `movie_path` at this file,
`plume_nav_sim` will ingest it into an internal dataset (Zarr) on demand the
first time you run the simulation. You never need to interact with the Zarr
store directly.

To refresh the dataset from the source movie, you can re-run the ingest CLI:

1. Regenerate or copy the AVI into this folder's `assets/` directory.
2. Convert it in-place to the internal dataset:

   ```bash
   conda run -n plume-nav-sim python -m plume_nav_sim.cli.video_ingest \
     --input plug-and-play-demo/assets/gaussian_plume_demo.avi \
     --output plug-and-play-demo/assets/gaussian_plume_demo.zarr \
     --fps 60 --pixel-to-grid "1 1" --origin "0 0" --normalize
   ```

The CLI uses the same contract as the automatic ingest used by the movie plume
field, so any dataset created this way will work with `plume="movie"` and
`movie_path` pointing at the AVI.

In [ ]:
# Save video helper (optional)
from typing import Sequence

import numpy as np


def save_video(
    frames: Sequence[np.ndarray], out_path: str = "demo.gif", fps: int = 10
) -> None:
    """Persist captured frames to disk using the library helper."""
    if not frames:
        print("No frames to save.")
        return
    try:
        from plume_nav_sim.utils.video import save_video_frames

        save_video_frames(frames, out_path, fps=fps)
        print(f"Saved video to {out_path}")
    except ImportError as exc:
        print("Install imageio to enable video export:", exc)
    except Exception as exc:  # pragma: no cover - notebook convenience
        print("Video export failed:", exc)

In [ ]:
import plume_nav_sim as pns

pns.get_package_info(include_environment_info=True)

> Prerequisite: install `plume_nav_sim` in your environment (e.g., `pip install plume_nav_sim` or editable install).

This notebook assumes `plume_nav_sim` is importable. The demo policy lives next to this notebook under the `plug_and_play_demo` package, so the dotted path resolves when running this notebook from its folder.

In [ ]:
# Ensure demo package is importable if running from repo root
import sys
from pathlib import Path

demo_root = Path.cwd()
if not (demo_root / "plug_and_play_demo").exists():
    candidate = demo_root / "plug-and-play-demo"
    if candidate.exists():
        sys.path.append(str(candidate))
        print("Added demo to sys.path:", candidate)
else:
    pass

In [ ]:
# Dev bootstrap: ensure backend config package is importable when running from the repo
import sys
from pathlib import Path

backend_root = Path.cwd() / "src" / "backend"
if backend_root.exists() and str(backend_root) not in sys.path:
    sys.path.insert(0, str(backend_root))
    print("Added backend to sys.path for config:", backend_root)

try:
    import config  # type: ignore[import-not-found]

    status = config.get_configuration_system_status()["system_status"]
    print("Config system status:", status)
except Exception as exc:  # pragma: no cover - notebook convenience
    # If this fails, the notebook will continue using fallback config implementations
    print("Config bootstrap failed; using fallback config:", exc)

## Set up the movie plume simulation

We will run the navigator inside the bundled movie plume dataset. The next cell builds a `SimulationSpec` that points at the Zarr store on disk and prepares `(env, policy)` via `prepare()` so we can stream an episode and capture frames.

In [ ]:
# Compose the simulation in the bundled movie plume (video file)
from pathlib import Path

from plume_nav_sim.compose import SimulationSpec, PolicySpec, WrapperSpec, prepare

MOVIE_FILE = Path("plug-and-play-demo/assets/gaussian_plume_demo.avi").resolve()
if not MOVIE_FILE.exists():
    raise FileNotFoundError(
        f"Movie file missing at {MOVIE_FILE}. See the instructions above for regenerating it."
    )

sim = SimulationSpec(
    grid_size=(128, 128),
    max_steps=1600,
    goal_radius=10.0,
    action_type="run_tumble",
    observation_type="concentration",
    reward_type="step_penalty",
    render=True,
    seed=123,
    plume="movie",
    movie_path=str(MOVIE_FILE),
    policy=PolicySpec(spec="plug_and_play_demo:DeltaBasedRunTumblePolicy"),
    observation_wrappers=[
        WrapperSpec(
            spec="plume_nav_sim.observations.history_wrappers:ConcentrationNBackWrapper",
            kwargs={"n": 2},
        )
    ],
)

env, policy = prepare(sim)
env.observation_space, env.action_space

In [ ]:
# Stream one episode and display frames (headless-safe)
from IPython.display import display, update_display
from PIL import Image

from plume_nav_sim.runner import runner

frames = []
actions = []
positions = []

disp = None
for ev in runner.stream(env, policy, seed=sim.seed, render=True):
    actions.append(int(ev.action))
    # Collect agent position from info (agent_xy preferred)
    pos = ev.info.get("agent_xy")
    if pos is not None:
        positions.append(pos)
    if ev.frame is not None:
        frames.append(ev.frame)
        img = Image.fromarray(ev.frame)
        if disp is None:
            disp = display(img, display_id=True)
        else:
            update_display(img, display_id=disp.display_id)
env.close()

len(frames), actions[:10]

In [ ]:
# Plot trajectory over a plume background
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from plume_nav_sim.envs.plume_search_env import unwrap_to_plume_env

%matplotlib inline

fig, ax = plt.subplots(figsize=(6, 6))
# Prefer last captured frame; if none, render a fresh one from the env
bg = None
if frames:
    bg = frames[-1]
else:
    try:
        base = unwrap_to_plume_env(env)
        try:
            bg = base.render("rgb_array")
        except TypeError:
            bg = base.render()
    except Exception:
        bg = None

if bg is not None:
    h, w = bg.shape[:2]
    ax.imshow(bg, origin="lower", extent=[0, w, 0, h])
if positions:
    xs, ys = zip(*positions)
    ax.plot(xs, ys, color="cyan", linewidth=2, alpha=0.9, label="path")
    ax.scatter([xs[0]], [ys[0]], c="yellow", s=40, label="start")
    ax.scatter([xs[-1]], [ys[-1]], c="red", s=40, label="end")
    if bg is not None:
        ax.set_xlim([0, w])
        ax.set_ylim([0, h])
        ax.set_aspect("equal", adjustable="box")
# Mark source and success radius if available
try:
    base = unwrap_to_plume_env(env)
    source = getattr(base, "source_location", None)
    goal_radius = getattr(base, "goal_radius", None)
    if source is not None:
        # source_location is a (x, y) tuple for PlumeSearchEnv
        try:
            sx, sy = source
        except Exception:
            sx, sy = source.x, source.y
        ax.scatter([sx], [sy], c="magenta", marker="x", s=60, label="source")
        if goal_radius is not None and goal_radius > 0:
            circle = patches.Circle(
                (sx, sy),
                radius=goal_radius,
                edgecolor="magenta",
                facecolor="none",
                linestyle="--",
                linewidth=2,
                alpha=0.7,
                label="success radius",
            )
            ax.add_patch(circle)
except Exception:
    pass
ax.set_title("Agent Trajectory")
ax.legend(loc="lower right")
plt.show()

In [ ]:
# Quick summary and artifact
summary = {"steps": len(actions), "frames": len(frames)}
print(summary)

# Persist a small artifact: GIF of the episode (optional)
save_video(frames, out_path="movie_demo.gif", fps=10)